## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
from utils import *

In [ ]:
img_name="timecity_01886_none"
img = cv2.imread(f'./svm_for_shadow_generation/images/{img_name}.jpg')
org_mask = cv2.imread(f'./svm_for_shadow_generation/masks/{img_name}.jpg')
data = json.load(open(f'./svm_for_shadow_generation/leg_pos/{img_name}.json', "r"))
fixed_points = np.zeros((1,4,2)) # leg positions
for i in range(4):
    fixed_points[0][i] = data['shapes'][i]['points'][0]

In [ ]:
is_match_legs = True
if is_match_legs is False:
    is_vertical_flip = np.random.choice(a=[False, True])
else:
    is_vertical_flip = False

if is_match_legs is True and is_vertical_flip is True:
    assert False, "Can not flip shadow while matching legs with TPS"

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
background=cv2.subtract(img,org_mask)
cut_obj=cv2.subtract(img,background)

In [ ]:
# Cut the mask so width == height to process with mask_to_shadow => get_rotate_matrix
# TODO: the rot_rd_high >30 might make the shadow too large compared to the original shadow
mask, cut_info = cut_mask(org_mask)
shadow_mask, rotate_matrix, trans_matrix = mask_to_shadow(mask, rot_rd_low=10,
                                                          rot_rd_high=20,
                                                          is_vertical_flip=is_vertical_flip,
                                                          match_legs=is_match_legs)
plt.imshow(shadow_mask[...,::-1])

In [ ]:
if match_legs: #using TPS
    while True:
        shadow_mask_out = match_legs(shadow_mask, fixed_points, rotate_matrix, -0.05, 0.05)
        l,r,t,b = get_mask_bbox(shadow_mask_out)
        if l>0 and t>0 and r<shadow_mask.shape[1]-1 and b<shadow_mask.shape[0]-1:
            shadow_mask = shadow_mask_out
            break
        else:
            print("shadow touch edges")
            break

# Recover original image size
pad_warped_shadow_mask = restore_mask(org_mask, shadow_mask, cut_info)
plt.imshow(pad_warped_shadow_mask[...,::-1])

In [ ]:
alpha = create_gradient_shadow_alpha_mask(pad_warped_shadow_mask, 
                                          rd_low=500, rd_high=580,
                                          is_vertical_flip=is_vertical_flip)
revert_shadow_mask=cv2.subtract(img,pad_warped_shadow_mask)

# Convert uint8 to float
revert_shadow_mask_f = revert_shadow_mask.astype(float)
img_f = img.astype(float)

# Multiply the foreground with the alpha matte
foreground_shadow = cv2.multiply(alpha, revert_shadow_mask_f)

# Multiply the background with ( 1 - alpha )
background_img = cv2.multiply(1.0 - alpha, img_f)

# Add the masked foreground and background.
shadow_on_top = cv2.add(foreground_shadow, background_img)
shadow_on_top = shadow_on_top.astype('uint8')

final_background=cv2.subtract(shadow_on_top,org_mask)
final_img = cut_obj+final_background
# plt.imshow(shadow_on_top[...,::-1])
plt.axis('off')
plt.imshow(final_img[...,::-1])

idx = "01"
cv2.imwrite(f'{img_name}_{idx}_merge.png', final_img)
cv2.imwrite(f'{img_name}_{idx}_alpha.png', alpha*255)
cv2.imwrite(f'{img_name}_{idx}_shadow_mask.png', pad_warped_shadow_mask)